# Feature Extraction for Deep Learning (WORD2VEC)

## Initialization

In [1]:
import pandas as pd
import numpy as np
import pickle

ICD9CODES = pickle.load(open("./data/ICD9CODES.p", "r"))

## WORD2VEC_DL_V0

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

STOPWORDS_WORD2VEC = stopwords.words('english') + ICD9CODES

def preprocessor_word2vec(text):
    text = re.sub('\[\*\*[^\]]*\*\*\]', '', text)
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower()) 
    text = re.sub(" \d+", " ", text)
    
    return text

def create_WORD2VEC_DL_V0(df, max_sequence_len=600, inputCol='text'):
    texts = df[inputCol].apply(preprocessor_word2vec)
    #texts = df['text']  # list of text samples

    toke = Tokenizer()
    toke.fit_on_texts(texts)
    sequence = toke.texts_to_sequences(texts)

    word_index = toke.word_index
    reverse_word_index = dict(zip(word_index.values(), word_index.keys())) # dict e.g. {1:'the', 2:'a' ...}
    #index_list = word_index.values()

    print('Found %s unique tokens.' % len(word_index))

    data = pad_sequences(sequence, maxlen=max_sequence_len)
    
    return data, word_index, reverse_word_index

def create_EmbeddingMatrix_V0(word_index, word2vec_model_path, remove_stopwords=True):

    embeddings_index = {}
    f = open(word2vec_model_path)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))
    
    if remove_stopwords:
        # Delete stopwords and ICD9 codes from pre-trained dictionary , 
        # so they will be zeros when we create embedding_matrix
        keys_updated = [word for word in embeddings_index.keys() if word not in STOPWORDS_WORD2VEC]
        index2word_set=set(keys_updated)
    else:
        index2word_set=set(embeddings_index.keys())
    
    EMBEDDING_DIM = embeddings_index.values()[0].size  # dimensions of the word2vec model

    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        if word in index2word_set: 
            #embedding_vector = embeddings_index.get(word)
        #if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embeddings_index.get(word)
            
    return embedding_matrix

Using TensorFlow backend.


## Actual Feature Extraction

In [10]:
import random, cPickle

def separate(seed, N):    
    idx=list(range(N))
    random.seed(seed)
    random.shuffle(idx)
    idx_train= idx[0:int(N*0.50)]
    idx_val= idx[int(N*0.50):int(N*0.75)]
    idx_test= idx[int(N*0.75):N]

    return idx_train, idx_val, idx_test

def separate_2(df, hadmid_pickle):
    f = open(hadmid_pickle, 'rb')
    hadmid_train = cPickle.load(f)
    hadmid_val = cPickle.load(f)
    hadmid_test = cPickle.load(f)
    f.close()
    
    df2 = df.copy()
    df2['_idx'] = df2.index
    df2.set_index('id', inplace=True)
    
    idx_train = df2.loc[hadmid_train]['_idx'].tolist()
    idx_val = df2.loc[hadmid_val]['_idx'].tolist()
    idx_test = df2.loc[hadmid_test]['_idx'].tolist()
    
    return idx_train, idx_val, idx_test

def batch_output_pickle(df, data, reversemap, fname, labels, hadmid_pickle='./data/TRAIN-VAL-TEST-HADMID.p'):
    idx_tuple = separate_2(df, hadmid_pickle)
    
    f = open(fname, 'wb')
    cPickle.dump(reversemap, f, protocol=cPickle.HIGHEST_PROTOCOL)
    for i in idx_tuple:
        cPickle.dump(data[i], f, protocol=cPickle.HIGHEST_PROTOCOL)
    for i in idx_tuple:
        cPickle.dump(df.loc[i][labels].values, f, protocol=cPickle.HIGHEST_PROTOCOL)        
    f.close()
    
def output_pickle(obj, fname):
    f = open(fname, 'wb')
    cPickle.dump(obj, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()

In [4]:
df = pd.read_csv("./data/DATA_HADM_TOP10.csv", escapechar='\\')
print df.head()

data, word_index, reverse_word_index = create_WORD2VEC_DL_V0(df, max_sequence_len=600)

       id  4019  2724  25000  4280  41401  53081  51881  42731  5849  5990  \
0  117760     0     0      0     0      0      1      1      0     0     0   
1  129030     1     1      0     0      0      1      0      0     0     0   
2  172040     0     0      0     0      1      0      0      0     1     0   
3  156170     0     0      1     1      0      0      0      1     1     0   
4  199180     0     0      1     1      1      0      0      0     0     0   

                                                text  
0  "Admission Date:  [**2118-12-14**]            ...  
1  Admission Date:  [**2137-8-31**]              ...  
2  Admission Date:  [**2174-1-6**]              D...  
3  Admission Date:  [**2102-6-9**]              D...  
4  Admission Date:  [**2164-7-2**]       Discharg...  
Found 117458 unique tokens.


In [11]:
labels = [`4019`, `2724`,`25000`,`4280`,`41401`,`53081`,`51881`,`42731`,`5849`,`5990`]
batch_output_pickle(df, data, reverse_word_index, "./data/DATA_WORDSEQV0_HADM_TOP10.p", labels)

In [6]:
em = create_EmbeddingMatrix_V0(word_index, "./data/model_word2vec.txt", remove_stopwords=True)
output_pickle(em, "./data/EMBMATRIX_WORD2VEC.p")
em = create_EmbeddingMatrix_V0(word_index, "./data/model_word2vec_v2_100dim.txt", remove_stopwords=True)
output_pickle(em, "./data/EMBMATRIX_WORD2VEC_v2_100dim.p")

Found 33837 word vectors.
Found 29939 word vectors.


Create ./data/TRAIN-VAL-TEST-HADMID.p

In [20]:
import cPickle

train = pd.read_csv("./data/DATA_TFIDFV1_HADM_TOP10_train.csv", escapechar='\\')
val = pd.read_csv("./data/DATA_TFIDFV1_HADM_TOP10_val.csv", escapechar='\\')
test = pd.read_csv("./data/DATA_TFIDFV1_HADM_TOP10_test.csv", escapechar='\\')

train2 = train['id'].tolist()
val2 = val['id'].tolist()
test2 = test['id'].tolist()

f = open('./data/TRAIN-VAL-TEST-HADMID.p', 'wb')
for obj in [train2, val2, test2]:
    cPickle.dump(obj, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()